2023-05-01 18:12:56.440192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
# %load train_svm
# %load train_svm
import os
import cv2
import joblib
import itertools
import numpy as np
from tqdm import tqdm
import mediapipe as mp
from pathlib import Path
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report

# augment.py
from augment import random_augmentations

# read images from these directories
images_list = list(Path("dataset").glob("**/*.png"))

# initialize face and mesh detector
face_detector = mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.5)
face_mesh = mp.solutions.face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")


labels_list = []
face_encodings_list = []

for file_path in tqdm(images_list):
    file_path = str(file_path)
    image = cv2.imread(file_path)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_resized = cv2.resize(image, [224, 224])

    augmented_images = random_augmentations(image_resized)

    for aug_image in augmented_images:
        results = face_detector.process(aug_image)

        if not results.detections:
            continue

        bbox = results.detections[0].location_data.relative_bounding_box
        xmin = int(bbox.xmin * aug_image.shape[1])
        ymin = int(bbox.ymin * aug_image.shape[0])
        width = int(bbox.width * aug_image.shape[1])
        height = int(bbox.height * aug_image.shape[0])
        face_image  = aug_image[ymin:height, xmin:width]

        if not face_image.size:
            continue

        face_blob = cv2.dnn.blobFromImage(face_image, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(face_blob)
        emeddings  = embedder.forward().flatten()

        face_encodings_list.append(emeddings)
        labels_list.append(file_path.split(os.path.sep)[-2])

        
        # results = face_mesh.process(aug_image)
        # if results.multi_face_landmarks:
        #     # landmark_values = []
        #     # for landmark in results.multi_face_landmarks[0].landmark:
        #     #     x = int(landmark.x * face_image.shape[1])
        #     #     y = int(landmark.y * face_image.shape[0])
        #     #     landmark_values.append(x)
        #     #     landmark_values.append(y)
                
        #     # face_encodings_list.append(np.array(landmark_values).flatten())
        #     labels_list.append(file_path.split(os.path.sep)[-2])

X = np.array(face_encodings_list)
y = np.array(labels_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

svm_model = svm.SVC(kernel='linear', C=1000, gamma=0.001, probability=True, verbose=2)
# svm_model = svm.SVC(kernel='rbf', C=10, gamma=0.001, probability=True)

print("fitting model...")
svm_model.fit(X_train, y_train)
print("done!")

y_pred = svm_model.predict(X_test)
print('Classification Report:\n', classification_report(y_test, y_pred))
print('Accuracy Score:', accuracy_score(y_test, y_pred))

print("saving the model and labels to file...")
joblib.dump((svm_model, label_encoder), "models/face_recognizer.sav")

# classes = svm_model.classes_
# cm = confusion_matrix(y_test, y_pred, labels=classes)

# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion matrix')
# plt.colorbar()
# tick_marks = np.arange(len(classes))
# plt.xticks(tick_marks, classes, rotation=45)
# plt.yticks(tick_marks, classes)
# fmt = 'd'
# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")


# fig, ax = plt.subplots(figsize=(8, 6))
# ax.plot(svm_model.decision_function(X_test))
# ax.set_xlabel('Samples')
# ax.set_ylabel('Confidence Scores')
# plt.show()

2023-05-02 02:44:00.897018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


4.6.0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
  7%|████████▊                                                                                                                           | 8/119 [00:02<00:29,  3.72it/s]

[ 0.08159881  0.01967803  0.00351933 -0.11301437  0.15851028  0.20283234
  0.08606265  0.08375849 -0.16902836  0.03115803  0.01996751  0.07160391
  0.1665872   0.0032993   0.09783096 -0.07578451 -0.00749871  0.03421636
 -0.09764177  0.14641248  0.14729801 -0.00572411  0.06856129  0.02577183
 -0.04364215 -0.03193959 -0.15759839 -0.112955    0.08583528  0.03735562
 -0.05247948 -0.01634657 -0.07866749  0.06815965  0.00540194  0.02509449
 -0.04243103  0.14545737 -0.04094798 -0.04010047  0.07608036  0.04098518
 -0.07183311  0.00944652 -0.24974753 -0.02020395  0.08524364  0.02409714
 -0.14956377  0.01621939 -0.03019375 -0.04783044 -0.13197966  0.0609921
  0.07448027 -0.00963895 -0.04461827  0.15393472 -0.11115743 -0.08471504
 -0.08329949  0.02224149  0.15211426 -0.12955876  0.13163808  0.00952808
  0.01098615  0.12309595 -0.08640968  0.00082527  0.11512429  0.1439925
 -0.00234852 -0.03497548  0.01681988  0.11260783 -0.09051949 -0.02253001
  0.12126715  0.01609345 -0.08974198  0.02955946  0.0

 13%|████████████████▌                                                                                                                  | 15/119 [00:03<00:14,  6.99it/s]

...

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 116/119 [00:21<00:00,  3.44it/s]

[-0.04517636  0.04183767  0.01587638 -0.14218573  0.07667273  0.0906873
  0.07172062  0.07515255 -0.02007236  0.09392013 -0.02313606  0.05627497
  0.06738291  0.17396712  0.11219514 -0.11971023  0.0605746  -0.01112841
  0.06445228  0.01950971  0.00171458  0.09029692  0.0266734   0.03704352
 -0.07089561 -0.03129388 -0.00105633  0.05623259  0.12683283  0.06015956
  0.00333616 -0.14018591  0.03922391 -0.03948994  0.03551096  0.0908427
 -0.01577292  0.12062452 -0.14380062  0.05403621 -0.059699    0.1507618
 -0.09443563  0.04331251 -0.16725239  0.04069847  0.00195729  0.01981064
  0.03816388  0.00935954 -0.02183055  0.09588012 -0.11611181 -0.02414818
  0.10463782  0.10158344 -0.07770428  0.19203296 -0.0744791   0.01947974
 -0.12722698 -0.02836912  0.1317934  -0.13173021  0.14038476 -0.09065937
  0.03861217  0.05552363 -0.10881399  0.15543301  0.16627449  0.12651159
 -0.01189128  0.00570635 -0.02480291  0.10119492 -0.02864357  0.03930254
  0.10751282 -0.13617207 -0.06295406  0.01698014  0.04

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:22<00:00,  5.31it/s]


[ 0.06853066  0.05721617 -0.0247639  -0.09337894  0.1701842   0.21902043
  0.09947948  0.06713109 -0.16914852  0.05806876  0.0190018   0.04360419
  0.15543634  0.03705633  0.13575915 -0.09423516  0.03120455  0.03610317
 -0.04970638  0.11642813  0.10745951 -0.01251509  0.08673172 -0.0023459
 -0.01134713 -0.07376594 -0.16666944 -0.0987106   0.11579383  0.0439391
 -0.01104284  0.01331641 -0.06316484  0.05089513  0.01105048  0.03731844
 -0.01618064  0.12728758 -0.02945356 -0.04454692  0.08634021  0.03744445
 -0.07647531 -0.01758611 -0.28825504 -0.00494643  0.06816552  0.04104052
 -0.12057808 -0.01683133 -0.04172688 -0.00592951 -0.1057654   0.03004126
  0.03788678  0.00985142 -0.02697969  0.15764403 -0.1040375  -0.05071427
 -0.12737554  0.03909217  0.14681527 -0.18386315  0.14837357  0.05362515
  0.01786012  0.1075992  -0.11125602  0.02339528  0.11169157  0.15780143
 -0.02158603 -0.02625634  0.02647632  0.10306308 -0.07045983 -0.03750679
  0.10299715 -0.01055446 -0.11148851  0.00980436  0.0

FileNotFoundError: [Errno 2] No such file or directory: 'models/face_recognizer.sav'